In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import time
import os
import psutil
import shelve
import pickle
from pybdm import BDM
from pybdm.utils import decompose_dataset
from joblib import Parallel, delayed
from joblib import parallel_backend
from data.algorithms import PerturbationExperiment, NodePerturbationExperiment
import math
import datetime
from itertools import product

In [ ]:
%pylab inline

## Visualization of database for ppt

In [2]:
PPI = pd.read_csv('data/original_data/bio-decagon-ppi.csv',sep=',')
PF = pd.read_csv('data/original_data/proteins.csv',sep=';')
DTI = pd.read_csv('data/original_data/bio-decagon-targets-all.csv',sep=',')
DDI = pd.read_csv('data/original_data/bio-decagon-combo.csv',sep=',')
DSE = pd.read_csv('data/original_data/bio-decagon-mono.csv',sep=',')

In [8]:
DSE.head()

,STITCH,Individual Side Effect,Side Effect Name
0,CID003062316,C1096328,central nervous system mass
1,CID003062316,C0162830,Photosensitivity reaction
2,CID003062316,C1611725,leukaemic infiltration brain
3,CID003062316,C0541767,platelet adhesiveness abnormal
4,CID003062316,C0242973,Ventricular dysfunction


## Modifying toy data to add drug features (fast)

In [ ]:
filename = './data/data_structures/DS_toy_genes16271_drugs639_se964'
with open(filename, 'rb') as f:
    DS = pickle.load(f)
    for key in DS.keys():
        globals()[key]=DS[key]
        print(key,"Imported successfully")

In [ ]:
n_genes,n_drugs = np.shape(dti_adj)
n_se = len(ddi_degrees_list)
print(n_genes,n_drugs,n_se)

In [ ]:
drug_feat = sp.csr_matrix(np.random.randint(0,2,(637,9702)))

In [ ]:
filename = './data/data_structures/DS_toy_DSE_genes'+str(n_genes)+'_drugs'+str(n_drugs)+'_se964'
data = {}
# DDI
data['ddi_adj_list'] = ddi_adj_list
data['ddi_degrees_list'] = ddi_degrees_list
# DTI
data['dti_adj'] = dti_adj
# PPI
data['ppi_adj'] = ppi_adj
data['ppi_degrees'] = ppi_degrees
# DSE
data['drug_feat'] = drug_feat
# PF
data['prot_feat'] = prot_feat

In [ ]:
with open(filename, 'wb') as f:
    pickle.dump(data, f, protocol=3)

## Modifying data to remove features

In [2]:
filename = './data/data_structures/DS_real_DSE_NPF_genes16271_drugs639_se964'
with open(filename, 'rb') as f:
    DS = pickle.load(f)
    for key in DS.keys():
        globals()[key]=DS[key]
        print(key,"Imported successfully")

gene2idx Imported successfully
drug2idx Imported successfully
se_mono_name2idx Imported successfully
se_combo_name2idx Imported successfully
ddi_adj_list Imported successfully
ddi_degrees_list Imported successfully
dti_adj Imported successfully
ppi_adj Imported successfully
ppi_degrees Imported successfully
drug_feat Imported successfully
prot_feat Imported successfully
norm_prot_feat Imported successfully


In [3]:
n_genes,n_drugs = np.shape(dti_adj)
n_se = len(ddi_degrees_list)
print(n_genes,n_drugs,n_se)

16271 639 964


In [ ]:
# Modify DSE and PF
drug_feat = sp.csr_matrix(np.random.randint(0,2,(n_drugs,n_drugs)))
prot_feat = sp.csr_matrix(np.random.randint(0,2,(n_genes,n_genes)))

In [4]:
# Modify only PF
prot_feat = sp.csr_matrix(np.random.randint(0,2,(n_genes,n_genes)))

In [6]:
# SAVING DATA STRUCTURES
filename = './data/data_structures/DS_real_DSE_genes'+str(n_genes)+'_drugs'+str(n_drugs)+'_se'+str(n_se)
data = {}
# Dictionaries
data['gene2idx'] = gene2idx
data['drug2idx'] = drug2idx
data['se_mono_name2idx'] = se_mono_name2idx
data['se_combo_name2idx'] = se_combo_name2idx
# DDI
data['ddi_adj_list'] = ddi_adj_list
data['ddi_degrees_list'] = ddi_degrees_list
# DTI
data['dti_adj'] = dti_adj
# PPI
data['ppi_adj'] = ppi_adj
data['ppi_degrees'] = ppi_degrees
# DSE
data['drug_feat'] = drug_feat
# PF
data['prot_feat'] = prot_feat
#data['norm_prot_feat'] = norm_prot_feat
with open(filename, 'wb') as f:
    pickle.dump(data, f, protocol=3)